In [1]:
import numpy as np
import collections
import os

path = os.path.abspath(os.getcwd())

from prep.derivatives import Preprocess_derivatives

from src.supplementary import Define_Derivatives
from src.term import normalize_ts, Term
from src.trainer import Equation_Trainer

Задаём функцию, позволяющую оценивать значений слагаемых уравнения. Это исполнение использует тензоры значений токенов на сетке и через поэлементное умножение тензоры, получает значения слагаемого. Далее, тензор преобразуется в вектор для использования в качестве признака в регрессии. 

In [2]:
def derivative_evaluator(term, eval_params):
    assert 'token_matrices' in eval_params and 'max_power' in eval_params
    if type(term) == Term:
        term = term.gene
    token_matrices = eval_params['token_matrices']
    value = np.copy(token_matrices[0])
    for var_idx in np.arange(eval_params['max_power'], term.shape[0], eval_params['max_power']):
        power = (np.sum(term[var_idx : var_idx + eval_params['max_power']]))
        value *= eval_params['token_matrices'][int(var_idx / float(eval_params['max_power']))] ** int(power)
    value = normalize_ts(value)
    value = value.reshape(np.prod(value.shape))
    return value    

Проводим препроцессинг данных и вычисляем значения производных на сетке. Вычисления довольно трудоёмкие => распараллелены, но всё равно могут занять относительно много времени, особенно если считать на ПК. Результаты препроцессинга сохраняем в отдельный файл, чтобы использовать при повторных запусках основного алгоритма. Это экономит время! Для примера используется решение волнового уравнения с белым шумом.
<br>
В этом примере мы рассмотрим задачу поиска уравнения по синтетическим данным, полученным из решения волнового уравнения: 
<br>
$\frac{\partial^2 u}{\partial t^2} = \frac{\partial^2 u}{\partial x_1^2} + \frac{\partial^2 u}{\partial x_2^2}$,
<br>
которое отражает эволюцию некоторой величины $u$ в двумерной области. Во входные данные, для большего сходства эксперимента с ситуацией из реальности, добавлен белый шум. 

In [ ]:
output_file_name = path + '/Preprocessing/Derivatives.npy'
filename =  path + '/Preprocessing/wave_noised.npy' 
poolsize = 4

field = np.load(filename)
Preprocess_derivatives(field, output_file_name, mp_poolsize=poolsize)

Загружаем значения в узлах сетки для исходной функции и её производных; из них формируем тензор для дальнейшего использования. Также задаём границы области, по которому будет вычисляться уравнение (в примере - обрезаем начало и конец временного ряда + по 15 элементов с каждой границы, чтобы использовать более "качественные" производные)

In [13]:
u_initial = np.load(path + '/Preprocessing/wave_noised.npy')
derivatives = np.load(path + '/Preprocessing/Derivatives.npy')
variables = np.ones((2 + derivatives.shape[1], ) + u_initial.shape)
variables[1, :] = u_initial
for i_outer in range(0, derivatives.shape[1]):
    variables[i_outer+2] = derivatives[:, i_outer].reshape(variables[i_outer+2].shape) 
    
skipped_elems = 10 
variables = variables[:, skipped_elems:-skipped_elems, skipped_elems:-skipped_elems, skipped_elems:-skipped_elems]

Получаем названия токенов для производных, используя функцию **Define_Derivatives()**. Она получает названия токенов в порядке: 1, u, $\frac{\partial u}{\partial x_1}$, $\frac{\partial^2 u}{\partial x_1^2}$, ... , $\frac{\partial u}{\partial x_2}$, $\frac{\partial^2 u}{\partial x_2^2}$, ...

In [9]:
token_names = Define_Derivatives(u_initial.ndim, max_order = 2)
print(token_names)

('1', 'u', 'du/dx1', 'd^2u/dx1^2', 'du/dx2', 'd^2u/dx2^2', 'du/dx3', 'd^2u/dx3^2')


Задаём объект для обучения уравнения:

In [5]:
Trainer = Equation_Trainer(token_list=token_names, evaluator = derivative_evaluator, 
                           evaluator_params={'token_matrices':variables})

Так как мы не знаем, какой параметр $\alpha$ - коэффициент регуляризации в LASSO, позволяет получить правильную структуру уравнения, мы проводим запуск по сетке из гиперпараметров модели. Сетка в примере строится только по одному параметру ($\alpha$), но в общем виде допустимо задавать сетки сразу по нескольким. Для остальные гиперпараметров модели задаём соответствующие значения. В случае, если каждый параметр задаётся одним значением, то их допустимо подавать сразу в метод Train.  

In [6]:
Trainer.Parameters_grid(('alpha', 'a_proc', 'r_crossover', 'r_mutation', 'mut_chance', 'pop_size', 'eq_len'), 
                        (0.001, 0.1, 3), 0.2, 0.6, 0.5, 1.0, 10, 6)

Запускаем обучение и получаем искомое уравнение в символьной форме

In [7]:
Trainer.Train(epochs = 200)

Using parameters from grid
in "process cell" function:  <class 'tuple'> ('1', 'u', 'du/dx1', 'd^2u/dx1^2', 'du/dx2', 'd^2u/dx2^2', 'du/dx3', 'd^2u/dx3^2')
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
Population for evolutionary algorithm initialized
Values of main selected parameters:
population size: 10 , r_crossover: 0.6 , r_mutation: 0.5
Alpha: 0.001
Achieved best fitness: 0.0132902017223155
Final gene: [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0132902017223155 {'1': 1, 'u': 0, 'du/dx1': 0, 'd^2u/dx1^2': 1, 'du/dx2': 0, 'd^2u/dx2^2': 0, 'du/dx3': 0, 'd^2u/dx3^2': 0}
weights: [ 0.         -0.14186451  0.51295421 -0.03489448  0.51288273]
result: ((datetime.datetime(2020, 2, 26, 16, 56, 49, 328405), datetime.datetime(2020, 2, 26, 17, 9, 25, 658232)), ({'1': 1, 'u': 0, 'du/dx1': 0, 'd^2u/dx1^2': 1, 'du/dx2': 0, 'd^2u/dx2^2': 0, 'du/dx3': 0, 'd^2u/dx3^2': 0}, [({'1': 1, 'u':

Получаем структуры уравнения в формате
<br>
$\frac{\partial^2 u}{\partial t^2} = a_2 \frac{\partial^2 u}{\partial x_1^2} + a_1 \frac{\partial^2 u}{\partial x_2^2} + F(u, \frac{\partial u}{\partial x_1}, \frac{\partial u}{\partial x_2}, ... )$, 
<br>
в котором помимо исходных слагаемых $\frac{\partial^2 u}{\partial t^2}$, $\frac{\partial^2 u}{\partial x_1^2}$ и $\frac{\partial^2 u}{\partial x_2^2}$, присутствует ещё и слагаемое $F(u, \frac{\partial u}{\partial x_1}, \frac{\partial u}{\partial x_2}, ... )$, в рассматриваемом случае соответствующее исходной функции $u$ из-за присутствия в данных шума. 

In [13]:
Trainer = Equation_Trainer(token_list=token_names, evaluator = derivative_evaluator, 
                           evaluator_params={'token_matrices':variables})
Trainer.Parameters_grid(('alpha', 'a_proc', 'r_crossover', 'r_mutation', 'mut_chance', 'pop_size', 'eq_len'), 
                        0.3, 0.2, 0.6, 0.5, 1.0, 10, 6)
Trainer.Train(epochs = 400)

Using parameters from grid
in "process cell" function:  <class 'tuple'> ('1', 'u', 'du/dx1', 'd^2u/dx1^2', 'du/dx2', 'd^2u/dx2^2', 'du/dx3', 'd^2u/dx3^2')
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Population for evolutionary algorithm initialized
Values of main selected parameters:
population size: 10 , r_crossover: 0.6 , r_mutation: 0.5
Alpha: 0.3
Achieved best fitness: 0.0031903655393910203
Final gene: [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0031903655393910203 {'1': 1, 'u': 0, 'du/dx1': 0, 'd^2u/dx1^2': 1, 'du/dx2': 0, 'd^2u/dx2^2': 0, 'du/dx3': 0, 'd^2u/dx3^2': 0}
weights: [ 0.         -0.08942311  0.         -0.         -0.        ]
result: ((datetime.datetime(2020, 2, 26, 19, 6, 30, 863834), datetime.datetime(2020, 2, 26, 19, 19, 17, 576416)), ({'1': 1, 'u': 0, 'du/dx1': 0, 'd^2u/dx1^2': 1, 'du/dx2': 0, 'd^2u/dx2^2': 0, 'du/dx3': 0, 'd^2u/dx3^2': 0}, [({'1': 1, 

При дальнейшем увеличении коэффициента регуляризации, уравнение теряет значимые слагаемые и принимает вид 
<br>
$\frac{\partial^2 u}{\partial t^2} = u$